In [3]:
import gym
import numpy as np
import random
from random import choices
import pickle
from pickle import load
from collections import Counter, deque
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.nn.utils as nn_utils
import torch.nn.functional as F
import torch.optim as optim

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [4]:
# Hyperparameters
batch_size = 32

gamma = 0.99

epsilon = 0.1

num_rounds = 50000
#num_episodes = 500
learning_limit = 100
replay_limit = 1000  # Number of steps until starting replay
#weight_update = 1000 # Number of steps until updating the target weights


In [5]:
#create model
class Model(nn.Module):
    def __init__(self, input_shape, n_actions):
        super(Model, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(input_shape[0]*input_shape[1], 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
            
        )

    def forward(self, x):
        # flatten the observation space Box to linear tensor
        tensor_array = torch.from_numpy(state)
        x_flat = torch.flatten(tensor_array).to(torch.float32)
        return self.net(x_flat)

In [6]:
class SchedulerEnv(gym.Env):

    def __init__(self):
        
        #starting parameters
        num_gps = 100
        num_slots = 40
        
#         num_pre_booked = 250
#         to_book = [2,1,2,2,1,1,1,3,3,1,2,1,3,2,1,1,2,1,3,2,3,2]
# #        to_book = [2,1,1,1,1]
#         num_to_book = len(to_book)
               
        num_pre_booked = random.randint(6*num_gps, 14*num_gps)
        num_to_book = random.randint(6*num_gps, 12*num_gps)
        to_book = []
        for j in range(num_to_book):
            to_book.append(*choices([1,2,3],[.7, .25, .05]))
                
        agent_pos = [0,0]
        
        #set parameters for the day
        self.num_gps = num_gps
        self.num_slots = num_slots
        self.num_pre_booked = num_pre_booked
        self.to_book = to_book
        self.num_to_book = num_to_book
        self.diary_slots = num_gps*num_slots
        self.agent_pos = agent_pos

        #set action space to move around the grid
        self.action_space = gym.spaces.Discrete(4) #up, down, left, right
        
        #set observation space 
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(self.num_slots, self.num_gps), dtype=np.int32)
   
    #creates daily diary for each gp, randomly populates prebooked appointments and resets parameters
    def reset(self):

        #creates zero filled dataframe with row per time slot and column per gp
        self.state = np.zeros((self.num_slots, self.num_gps),dtype=float)

        #randomly enters a 1 for each pre booked appointments
        pre_booked = self.num_pre_booked
        while pre_booked>0:
            pre_booked -= 1
            self.state[np.random.randint(self.num_slots), np.random.randint(self.num_gps)] = 1
            
        #randomly sets the agent start space
        self.agent_pos = [np.random.randint(self.num_slots), np.random.randint(self.num_gps)]

        #resets parameters for new episode
        self.done = False
        self.reward = 0
        self.appt_idx = 0
        
        #print('starting state', self.state.sum(), self.state)

        return self.state
    
    #calculates new position of the agent based on the action
    def move_agent(self, action):

        #set boundaries for the grid
        max_row = self.num_slots - 1
        max_col = self.num_gps - 1

        #setting new co-ordinates for the agent
        new_row = self.agent_pos[0]
        new_col = self.agent_pos[1]

        #calculate what the new position may be based on the action without going out the grid
        if action == 0:
            #print('up')
            new_row = max(self.agent_pos[0] - 1, 0)
        if action == 1:
            #print('down')
            new_row = min(self.agent_pos[0] + 1, max_row)
        if action == 2:
            #print('left')
            new_col = max(self.agent_pos[1] - 1, 0)
        if action == 3:
            #print('right')
            new_col = min(self.agent_pos[1] + 1, max_col)

        new_pos = [new_row, new_col]
        #print('new pos', new_pos)

        return new_pos

    #checks if we can look to book appointment starting here
    def check_bookable(self):
        return self.state[self.agent_pos[0], self.agent_pos[1]] == 0.0
    
    #action if we can't book the appointment
    def invalid_booking(self):
        #print('cant book')
        self.reward = -1
        
    #action if we can book the appointment
    def valid_booking(self):
        #print('go ahead and book')
        self.appt_idx += 1
        self.reward = 1
    
    #checks if the appointment fits
    def check_and_book(self):
        
        max_row = self.num_slots - 1
        cells_to_check = self.to_book[self.appt_idx]
        
        if cells_to_check==1:
            #print('good to check for single')
            if self.state[self.agent_pos[0], self.agent_pos[1]] == 0:
                self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                self.valid_booking()
            else:
                #print('single taken')
                self.invalid_booking()

        if cells_to_check==2:
            #check we're not at the bottom of the grid
            if self.agent_pos[0]<max_row:
                #check the next cells is also 0.0
                #print('good to check for double')
                if self.state[self.agent_pos[0], self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+1), self.agent_pos[1]] == 0:
                    self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+1), self.agent_pos[1]] = 1
                    self.valid_booking()
                    self.agent_pos = [(self.agent_pos[0]+1), self.agent_pos[1]]
                    #print('after booking', self.agent_pos)
                else:
                    #print('double taken')
                    self.invalid_booking()
            else:
                #print('not for double')
                self.invalid_booking()
                
        if cells_to_check==3:
            #check we're not at the bottom of the grid
            if self.agent_pos[0]+1<max_row:
                #print('good to check for treble')
                if self.state[self.agent_pos[0], self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+1), self.agent_pos[1]] == 0 \
                 and self.state[(self.agent_pos[0]+2), self.agent_pos[1]] == 0:
                    self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+1), self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+2), self.agent_pos[1]] = 1
                    self.valid_booking()
                    self.agent_pos = [(self.agent_pos[0]+2), self.agent_pos[1]]
                else:
                    #print('treble taken')
                    self.invalid_booking()
            else:
                #print('not for treble')
                self.invalid_booking()
                
        if cells_to_check==4:
            #check we're not at the bottom of the grid
            if self.agent_pos[0]+2<max_row:
                #check the next cells is also 0.0
                #print('good for quad')
                if self.state[self.agent_pos[0], self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+1), self.agent_pos[1]] == 0 \
                 and self.state[(self.agent_pos[0]+2), self.agent_pos[1]] == 0 and \
                self.state[(self.agent_pos[0]+3), self.agent_pos[1]] == 0:
                    self.state[self.agent_pos[0], self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+1), self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+2), self.agent_pos[1]] = 1
                    self.state[(self.agent_pos[0]+3), self.agent_pos[1]] = 1
                    self.valid_booking()
                    self.agent_pos = [(self.agent_pos[0]+3), self.agent_pos[1]]
                else:
                    #print('quad taken')
                    self.invalid_booking()
            else:
                #print('not for quad')
                self.invalid_booking()

        next_state = self.state

        return next_state

    def step(self, action):

        #get new position of agent based on action
        new_agent_pos = self.move_agent(action)
        #print('new and old pos', new_agent_pos, self.agent_pos)
        
        #if the agent is stuck on an edge then move to a new position
        if new_agent_pos == self.agent_pos:
            self.agent_pos = [np.random.randint(self.num_slots), np.random.randint(self.num_gps)]
            #print('here1', self.agent_pos)
        else:
            self.agent_pos = new_agent_pos
            #print('here2', self.agent_pos)
            
        #print('trying to book', self.to_book, self.appt_idx)
        
        #check if it's possible to book then book
        if self.check_bookable():
            #print('checked here')
            self.state = self.check_and_book()
        else:
            #print('not bookable')
            self.invalid_booking()
        
        #work out if episode complete
        if self.appt_idx == len(self.to_book):
            self.done = True
            
        #print(self.state, self.agent_pos)
        agent_state = self.state.copy()
        agent_state[self.agent_pos[0], self.agent_pos[1]] = 5
        #print('agent', agent_state)

        info = {}
        return agent_state, self.reward, self.done, info

In [7]:
env = SchedulerEnv()

In [ ]:
#device = "cuda"
device = "cpu"

env = SchedulerEnv()

#start writing to tensorboard
writer = SummaryWriter(comment="Scheduler MC")

#create the current network and target network
policy_model = Model(env.observation_space.shape, env.action_space.n).to(device)

optimizer = optim.Adam(policy_model.parameters(), lr=0.001, eps=1e-3)
stp_idx = 0

#epsilon = eps_start
for a in range(num_rounds):
    #print(a)
    episode_list =[]

    for i in range(10):
        #change this for while not true once it works
        state = env.reset()
        episode_reward = 0
        done = False
        #print('reset here')

        for j in range(50):
    #    while not done:

            # Select and perform an action
            if np.random.rand() > epsilon:
                action = torch.argmax(policy_model(state))
            else:
                action = np.random.randint(env.action_space.n)

            next_state, reward, done, _ = env.step(action)
            episode_reward += reward

            episode_list.append([state, action, reward, done])

            # Move to the next state
            state = next_state
            
            stp_idx +=1
            if stp_idx%100000 == 0:
                print('stp_idx', stp_idx)
            
            if done:
                break

        #print('stopped episode', j, episode_reward)

        writer.add_scalar('episode_reward', episode_reward, stp_idx)

        #print('step', step_idx, 'i', i, 'j', j, episode_reward)

    #create list of state, action pairs with values
    #set list to capture actual rewards per step
    tot_reward = []
    #set sum to zero
    reward_to_add = 0
    #work backwards through each step
    for item in reversed(episode_list):
        #if the step is a final step
        if item[3]:
            #append the step and actual reward to the list
            tot_reward.append([item[0], int(item[1]), item[2]])
            #set the start for the cumulative reward
            reward_to_add = 1
        #if the step is not a final step
        else:
            #add the reward for this step to the cumulative total
            reward_to_add += item[2]
            #append the step with the cumulative total to the list
            tot_reward.append([item[0], int(item[1]), reward_to_add])

    # create dictionary to be able to average action values to use for training
    sa_dict = {}
    #loop through all the steps
    for item in tot_reward:
        #flatten each state into a tuple
        flat_state = tuple(item[0].flatten())
        #set the action to an integer
        int_act = int(item[1])
        k = (flat_state, int_act)
        #if the state action pair is in the dictionary
        if k in sa_dict.keys():
            #append the reward
            sa_dict[k].append(item[2])
        else:
            #create a new entry for the state action pair
            sa_dict[k] = [item[2]] 

    optimizer.zero_grad()

    #create empty lists to record the states, actions and rewards
    state_list = []
    action_list = []
    state_value = []
    #loop through each key in the dictionary
    for key in sa_dict.keys():
        #append ths state and action to the respective list
        state_list.append(key[0])
        action_list.append(key[1])
        v = sa_dict[key]
        #if the total sum of the rewards is zero, append to the reward list
        if sum(v) == 0:
            state_value.append(sum(v))
        #else work out the average and append this to the reward list
        else:
            state_value.append(sum(v)/ float(len(v))) 

    predicted = []
    target = []
    for i in range(len(state_list)):
        predicted.append(policy_model(state_list[i])[action_list[i]])
        target.append(state_value[i])
    
    #print('this stp_idx', stp_idx)

    loss = F.mse_loss(torch.Tensor(predicted), torch.Tensor(target))
    loss.requires_grad = True
    loss.backward()
    writer.add_scalar('loss', loss, stp_idx)

    optimizer.step()      

    if stp_idx>3000000:
        break

writer.close()

In [8]:
filename = 'finalized_model.sav'
#pickle.dump(policy_model, open(filename, 'wb'))

In [9]:
state = env.reset()

In [10]:
loaded_model = pickle.load(open(filename, 'rb'))

In [11]:
#policy_model(my_state[2])

In [13]:
loaded_model(state)

tensor([-0.0037, -0.0202, -0.0544,  0.0982], grad_fn=<AddBackward0>)

In [15]:
test_out = torch.argmax(loaded_model(state))
test_out

tensor(3)

In [16]:
import pandas as pd

In [17]:
#read in the data from the futuresNHS and select the day to schedule
appt_figs = pd.read_csv('JanData.csv')
day_to_schedule = appt_figs[appt_figs['Date'].isin(['11-Jan-22'])].to_numpy().flatten()

In [18]:
#calculate the national appointment split for the day
tot_appt = float((day_to_schedule[1]).replace(',',''))
to_book = float((day_to_schedule[7]).replace(',',''))
pre_book = tot_appt-to_book

In [19]:
#calculate the percentage split of appointment type
small_app = (float((day_to_schedule[2]).replace(',',''))+float((day_to_schedule[5]).replace(',','')))/tot_appt
med_app = float((day_to_schedule[4]).replace(',',''))/tot_appt
large_app = 1-small_app-med_app

In [20]:
#calculate the appointments due for 100 gps
surgery_to_book = round(to_book/float((day_to_schedule[8]).replace(',',''))*100)
surgery_pre_booked = round(pre_book/float((day_to_schedule[8]).replace(',',''))*100)

In [21]:
#create the scheduler with pre booked appointments
def create_starting_schedule(num_gps, num_slots, num_pre_booked): 
    
    #creates zero filled dataframe with row per time slot and column per gp
    state = np.zeros((num_slots, num_gps),dtype=float)

    #randomly enters a 1 for each pre booked appointments
    while num_pre_booked>0:
        num_pre_booked -= 1
        state[np.random.randint(num_slots), np.random.randint(num_gps)] = 1
    
    return state

In [22]:
#create list of appointments to book based on split
def list_to_book(num_to_book, small_split, med_split, large_split):
    to_book = []
    for i in range(num_to_book):
        to_book.append(*choices([1,2,3],[small_split, med_split, large_split]))
    return to_book

In [23]:
#calculates new position of the agent based on the action
def move_agent(agent_pos, action):

    #set boundaries for the grid
    max_row = num_slots - 1
    max_col = num_gps - 1
    
    #setting new co-ordinates for the agent
    new_row = agent_pos[0]
    new_col = agent_pos[1]

    #calculate what the new position may be based on the action without going out the grid
    if action == 0:
        new_row = max(agent_pos[0] - 1, 0)
    if action == 1:
        new_row = min(agent_pos[0] + 1, max_row)
    if action == 2:
        new_col = max(agent_pos[1] - 1, 0)
    if action == 3:
        new_col = min(agent_pos[1] + 1, max_col)

    new_pos = [new_row, new_col]

    return new_pos

In [24]:
#checks if the appointment fits
def check_and_book(state, cells_to_check, agent_pos, appt_idx):

    max_row = num_slots - 1

    if cells_to_check==1:
        if state[agent_pos[0], agent_pos[1]] == 0:
            state[agent_pos[0], agent_pos[1]] = 1
            appt_idx+=1

    if cells_to_check==2:
        #check we're not at the bottom of the grid
        if agent_pos[0]<max_row:
            if state[agent_pos[0], agent_pos[1]] == 0 and \
            state[(agent_pos[0]+1), agent_pos[1]] == 0:
                state[agent_pos[0], agent_pos[1]] = 1
                state[(agent_pos[0]+1), agent_pos[1]] = 1

                agent_pos = [(agent_pos[0]+1), agent_pos[1]]
                appt_idx+=1

    if cells_to_check==3:
        #check we're not at the bottom of the grid
        if agent_pos[0]+1<max_row:
            if state[agent_pos[0], agent_pos[1]] == 0 and \
            state[(agent_pos[0]+1), agent_pos[1]] == 0 \
             and state[(agent_pos[0]+2), agent_pos[1]] == 0:
                state[agent_pos[0], agent_pos[1]] = 1
                state[(agent_pos[0]+1), agent_pos[1]] = 1
                state[(agent_pos[0]+2), agent_pos[1]] = 1

                agent_pos = [(agent_pos[0]+2), agent_pos[1]]
                appt_idx+=1

    next_state = state

    return next_state, agent_pos, appt_idx

In [25]:
num_slots = 40
num_gps = 100
state = create_starting_schedule(num_gps, num_slots, surgery_pre_booked)
to_book = list_to_book(surgery_to_book, small_app, med_app, large_app)
agent_pos = (0,0)
appt_idx=0

In [27]:
#while appt_idx < len(test_to_book):
for i in range(5):
    cells_to_check = to_book[appt_idx]
    state, agent_pos, appt_idx = check_and_book(state, cells_to_check, agent_pos, appt_idx) 
    agent_state = state.copy()
    agent_state[agent_pos[0], agent_pos[1]] = 5
    action = torch.argmax(loaded_model(agent_state))
    agent_pos = move_agent(agent_pos, action)
    print(appt_idx, action, state, agent_pos)

1 tensor(3) [[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [1, 1]
2 tensor(0) [[1. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [1, 1]
2 tensor(0) [[1. 0. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [0, 1]
3 tensor(3) [[1. 1. 0. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [0, 2]
4 tensor(0) [[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [0, 2]
